In [1]:
from sklearn import *
from tensorflow import keras as keras
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.datasets import load_digits
from tensorflow.keras.utils import to_categorical
import pandas as pd
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from tensorboard.plugins import projector

root_dir = r'/Users/pankaj/Library/Mobile Documents/com~apple~CloudDocs/Capstone/Wikipedia Data'


result_path = annot_file_path = os.path.join(root_dir , 'comments_with_grouped_annoptations.tsv')

merged_comments = pd.read_table(result_path)


In [2]:
merged_comments['recipient_attack'] = merged_comments['recipient_attack'].apply(lambda x : 1 if x> 1 else 0 )
X_train = merged_comments['comment']
y = merged_comments['recipient_attack']
one_hot_train_labels = to_categorical(y)

In [3]:
merged_comments['new_attack'] = merged_comments['attack'].apply(lambda x : 1 if x> 1 else 0 )

### TODO change model with proper word settings 

In [5]:
# Prepare training input  
training_samples = 90000
validation_samples = 10000
max_words = 15000
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)
sequences = tokenizer.texts_to_sequences(X_train)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))


Found 187899 unique tokens.


In [6]:
maxlen = 150

In [7]:
data = pad_sequences(sequences, maxlen=maxlen)
print('Shape of data tensor:', data.shape)
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
x_train = data[:training_samples]
x_val = data[training_samples: (training_samples + validation_samples)]
x_test = data[(training_samples + validation_samples) : ]

Shape of data tensor: (115864, 150)


In [8]:
# Prepare labels 
labels = np.asarray(one_hot_train_labels)
print('Shape of label tensor:', labels.shape)
labels = labels[indices]
y_train = labels[:training_samples]
y_val = labels[training_samples: training_samples + validation_samples]
y_test = labels[(training_samples + validation_samples) : ]

Shape of label tensor: (115864, 2)


In [11]:
from tensorflow.keras.layers import LSTM
embedding_dim = 16
model = Sequential()
#embedding = Embedding(max_words, embedding_dim , weights =w)
embedding = Embedding(max_words, embedding_dim)

#embedding.trainable = False
model.add(embedding)
model.add(LSTM(32))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(2, activation='sigmoid'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 16)          240000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                6272      
_________________________________________________________________
flatten_1 (Flatten)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 66        
Total params: 247,394
Trainable params: 247,394
Non-trainable params: 0
_________________________________________________________________


In [12]:
model.compile(optimizer='rmsprop',
loss='binary_crossentropy',
metrics=['acc'])
history = model.fit(x_train, y_train,
epochs= 2,
batch_size=128,
validation_data=(x_val, y_val))

Train on 90000 samples, validate on 10000 samples
Epoch 1/2
90000/90000 [==============================] - 81s 902us/sample - loss: 0.3725 - acc: 0.8441 - val_loss: 0.3231 - val_acc: 0.8702
Epoch 2/2
90000/90000 [==============================] - 80s 893us/sample - loss: 0.3013 - acc: 0.8768 - val_loss: 0.3185 - val_acc: 0.8707


In [ ]:
model.save('basic_lstm_with_auto_encoder.h5')

In [ ]:
word_index_1 = {k, v+3 for k, v in word_index.items()}
word_index_1['<padding>'] = 0
word_index_1['<start>'] = 1
word_index_1['<unknown>'] = 2

counter = 3



In [14]:
word_index

{'the': 1,
 'newline': 2,
 'to': 3,
 'tokennewline': 4,
 'and': 5,
 'of': 6,
 'a': 7,
 'you': 8,
 'token': 9,
 'i': 10,
 'is': 11,
 'that': 12,
 'in': 13,
 'it': 14,
 'for': 15,
 'not': 16,
 'this': 17,
 'on': 18,
 'be': 19,
 'are': 20,
 'as': 21,
 'have': 22,
 'your': 23,
 'with': 24,
 'was': 25,
 'or': 26,
 'article': 27,
 'if': 28,
 'but': 29,
 'my': 30,
 'an': 31,
 'by': 32,
 'wikipedia': 33,
 'page': 34,
 'from': 35,
 'do': 36,
 'me': 37,
 'about': 38,
 'at': 39,
 'so': 40,
 'all': 41,
 'what': 42,
 'can': 43,
 'will': 44,
 'has': 45,
 'no': 46,
 'there': 47,
 'he': 48,
 'like': 49,
 'they': 50,
 'one': 51,
 'would': 52,
 'just': 53,
 'which': 54,
 'who': 55,
 'been': 56,
 'any': 57,
 "don't": 58,
 'should': 59,
 'his': 60,
 'we': 61,
 'more': 62,
 'talk': 63,
 'some': 64,
 'other': 65,
 'please': 66,
 'am': 67,
 'here': 68,
 'think': 69,
 'know': 70,
 'because': 71,
 'see': 72,
 'people': 73,
 'also': 74,
 'up': 75,
 'fuck': 76,
 'only': 77,
 'how': 78,
 'tokeni': 79,
 'out': 80,

In [29]:
def preprocess_text(text_eval):
    sequences = tokenizer.texts_to_sequences([text_eval])
    data = pad_sequences(sequences, maxlen=maxlen)
    return data

In [30]:
d = preprocess_text("white card pankaj wq adqwd dqwwq ") 

In [32]:
len(d[0])

150

In [44]:
x_train_1 = merged_comments[:training_samples]

In [50]:
x_train_1.head()

,0,1,2,3,4,5,6,7,8,9,...,140,141,142,143,144,145,146,147,148,149
0,0,0,0,0,0,0,0,0,0,0,...,96,2064,11,24,57,6,1,891,2264,1935
1,0,0,0,0,0,0,0,0,0,0,...,1,3,4795,815,442,1,1850,1230,103,820
2,0,0,0,0,0,0,0,0,0,0,...,5,86,61,20,1,287,5,8,1077,1963
3,0,0,0,0,0,0,0,0,0,0,...,1209,37,42,1,729,17,11,2,4,9
4,0,0,0,0,0,0,0,0,0,0,...,2048,24,62,7023,35,698,72,63,5876,13376


In [55]:
x_train_1 = merged_comments[:training_samples]
x_val_1 = merged_comments[training_samples: (training_samples + validation_samples)]
x_test_1 = merged_comments[(training_samples + validation_samples) : ]

In [52]:
x_train_1.head()

,Unnamed: 0,rev_id,comment,year,logged_in,ns,sample,split,worker_id,quoting_attack,recipient_attack,third_party_attack,other_attack,attack,new_attack
0,0,37675,`-NEWLINE_TOKENThis is not ``creative``. Thos...,2002,False,article,random,train,9721,0.0,0,0.0,0.0,0.0,0
1,1,44816,`NEWLINE_TOKENNEWLINE_TOKEN:: the term ``stand...,2002,False,article,random,train,20234,0.0,0,0.0,0.0,0.0,0
2,2,49851,"NEWLINE_TOKENNEWLINE_TOKENTrue or false, the s...",2002,False,article,random,train,26474,0.0,0,0.0,0.0,0.0,0
3,3,89320,"Next, maybe you could work on being less cond...",2002,True,article,random,dev,26738,0.0,1,0.0,2.0,4.0,1
4,4,93890,This page will need disambiguation.,2002,True,article,random,train,8010,0.0,0,0.0,0.0,0.0,0


In [70]:
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords 
import tensorflow_hub as hub


In [60]:
 pred_vlaues = model.predict(preprocess_text(str("dark night sas")))

In [61]:
pred_vlaues

array([[0.89783835, 0.09717047]], dtype=float32)

In [71]:
module_url = "https://tfhub.dev/google/elmo/3"
embed = hub.load(module_url)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [74]:
NAE_vals_array = np.empty((3,1024), float)
# NAE_vals_array = []
PAE_vals_array = np.empty((3,1024), float)

for index, row in x_train_1.iterrows():
    print(" running count {}".format(index))
    text = row.comment.translate(str.maketrans('','',string.punctuation))
    tokens = word_tokenize(text)
    tokens_without_sw = [word for word in tokens if not word in stopwords.words()]
   
    ### First Get the overall Classification
    pred_vlaues = model.predict(preprocess_text(str(tokens_without_sw)))
    classification = pred_vlaues.argmax()
    
    if classification == 0 and row.recipient_attack == 0:
        mode = 'PAE' # Positive AutoEncoder
    elif classification == 0 and row.recipient_attack == 1:
        mode = 'NAE'
    elif classification == 1 and row.recipient_attack == 0:
        mode = 'PAE'
    elif classification == 1 and row.recipient_attack == 1:
        mode = 'NAE'
    ### Get the Elmo vector
    text=tf.convert_to_tensor([str(tokens_without_sw)])
    out = embed.signatures['default'](text)['elmo']
    pred_vals_array = np.empty((0,2), float)
    
    for i in range(len(tokens_without_sw)-2):
        text_eval = tokens_without_sw[i:i+3]
        a = preprocess_text(str(text_eval))
        pred_vlaues = model.predict(a)
        pred_vals_array = np.vstack((pred_vals_array,pred_vlaues))
    if mode == "NAE":
        # Only Negative Vals, and find the index of the highest contributing phrase
        phrase_start_index = pred_vals_array[:,1].argmax()
        ### Get the ELMO Encoding of the entire vector:
        elmo_vecs = np.array(out[0][phrase_start_index:phrase_start_index+3])
        NAE_vals_array = np.vstack((NAE_vals_array, elmo_vecs))
    elif mode == "PAE":
        # Only Positive Vals, and find the index of the highest contributing phrase
        phrase_start_index = pred_vals_array[:,0].argmax()
        PAE_vals_array = np.vstack((PAE_vals_array,out[0][phrase_start_index:phrase_start_index+3]))
    if index == 10:
        break

 running count 0
 running count 1
 running count 2
 running count 3
 running count 4
 running count 5
 running count 6
 running count 7
 running count 8
 running count 9
 running count 10


In [54]:
row

0         0
1         0
2         0
3         0
4         0
       ... 
145       6
146       1
147     891
148    2264
149    1935
Name: 0, Length: 150, dtype: int32

In [76]:
NAE_vals_array = NAE_vals_array.reshape(int(NAE_vals_array.shape[0]/3),3,1024)


In [77]:
# lstm autoencoder to recreate a timeseries
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import RepeatVector
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.callbacks import EarlyStopping

In [78]:
timesteps = 3
n_features = 1024
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)

# define model
model_TAE = Sequential()
model_TAE .add(LSTM(1024, activation='relu', input_shape=(timesteps,n_features), return_sequences=True))
model_TAE .add(LSTM(720, activation='relu', input_shape=(timesteps,n_features), return_sequences=True))
model_TAE .add(LSTM(128, activation='relu', input_shape=(timesteps,n_features), return_sequences=True))
model_TAE .add(LSTM(64, activation='relu', return_sequences=False))
model_TAE .add(RepeatVector(timesteps))
model_TAE .add(LSTM(64, activation='relu', return_sequences=True))
model_TAE .add(LSTM(128, activation='relu', return_sequences=True))
model_TAE .add(LSTM(720, activation='relu', return_sequences=True))
model_TAE .add(LSTM(1027, activation='relu', return_sequences=True))
model_TAE .add(TimeDistributed(Dense(n_features)))
model_TAE .compile(optimizer='adam', loss='mse')
model_TAE.summary()
# model_TAE .fit(aa, aa, epochs=3, batch_size=5)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 16)          240000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                6272      
_________________________________________________________________
flatten_1 (Flatten)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 66        
Total params: 247,394
Trainable params: 247,394
Non-trainable params: 0
_________________________________________________________________


In [80]:
model_TAE.summary()
model_TAE.fit(NAE_vals_array, NAE_vals_array, epochs=3, batch_size=5)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 3, 1024)           8392704   
_________________________________________________________________
lstm_3 (LSTM)                (None, 3, 720)            5025600   
_________________________________________________________________
lstm_4 (LSTM)                (None, 3, 128)            434688    
_________________________________________________________________
lstm_5 (LSTM)                (None, 64)                49408     
_________________________________________________________________
repeat_vector (RepeatVector) (None, 3, 64)             0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 3, 64)             33024     
_________________________________________________________________
lstm_7 (LSTM)                (None, 3, 128)           